## Create TICKIT Table

In [0]:
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS venue;
DROP TABLE IF EXISTS category;
DROP TABLE IF EXISTS dwdate;
DROP TABLE IF EXISTS event;
DROP TABLE IF EXISTS sales;

create table users(
	userid integer not null,
	username char(8),
	firstname varchar(30),
	lastname varchar(30),
	city varchar(30),
	state char(2),
	email varchar(100),
	phone char(14),
	likesports boolean,
	liketheatre boolean,
	likeconcerts boolean,
	likejazz boolean,
	likeclassical boolean,
	likeopera boolean,
	likerock boolean,
	likevegas boolean,
	likebroadway boolean,
	likemusicals boolean);

create table venue(
	venueid smallint not null,
	venuename varchar(100),
	venuecity varchar(30),
	venuestate char(2),
	venueseats integer);

create table category(
	catid smallint not null,
	catgroup varchar(10),
	catname varchar(10),
	catdesc varchar(50));

create table dwdate(
	dateid smallint not null,
	caldate date not null,
	day character(3) not null,
	week smallint not null,
	month character(5) not null,
	qtr character(5) not null,
	year smallint not null,
	holiday boolean default('N'));

create table event(
	eventid integer not null,
	venueid smallint not null,
	catid smallint not null,
	dateid smallint not null,
	eventname varchar(200),
	starttime timestamp);

create table listing(
	listid integer not null,
	sellerid integer not null,
	eventid integer not null,
	dateid smallint not null,
	numtickets smallint not null,
	priceperticket decimal(8,2),
	totalprice decimal(8,2),
	listtime timestamp);


create table sales(
	salesid integer not null,
	listid integer not null,
	sellerid integer not null,
	buyerid integer not null,
	eventid integer not null,
	dateid smallint not null,
	qtysold smallint not null,
	pricepaid decimal(8,2),
	commission decimal(8,2),
	saletime timestamp);

## COPY Data From S3

In [0]:
copy users from 's3://cdk-entest-videos/tickit/allusers_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy venue from 's3://cdk-entest-videos/tickit/venue_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy category from 's3://cdk-entest-videos/tickit/category_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy dwdate from 's3://cdk-entest-videos/tickit/date2008_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy event from 's3://cdk-entest-videos/tickit/allevents_pipe.txt'
iam_role default
delimiter '|' timeformat 'YYYY-MM-DD HH:MI:SS' region 'us-east-1';

copy listing from 's3://cdk-entest-videos/tickit/listings_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy sales from 's3://cdk-entest-videos/tickit/sales_tab.txt'
iam_role default
delimiter '\t' timeformat 'MM/DD/YYYY HH:MI:SS' region 'us-east-1';

## Query Plan Explain - JOIN 
Look at the explain, DS_BCAST_INNER (Distribution: Broadcast Inner Table) which indicate that a copy of the entire table (the listing table) is broadcast to all of the compute nodes. This occurs because the data for both tables must be brought together on the same slice to join each given row during the query. 

The cost field provides a relative measure of the effort required to run the query. It is measured in cost units that are arbitrary but conventially mean disk page fetches. Basically, a smaller number is better. 

The cost in this query plan is very large (124168336397.54) indicating that a lot of data is moving around the Amazon Redshift cluster. This query can be optimized by selecting a distribution key that co-locate rows from both tables on the same slice where they have a common value in the selected KEY column. Based on the below query, it appears that the listid column is the most-commonly joined column, so this should be selected as the DISTKEY

- [join syntax](https://docs.aws.amazon.com/redshift/latest/dg/r_FROM_clause30.html#r_FROM_clause30-parameters)
- [join example](https://docs.aws.amazon.com/redshift/latest/dg/r_Join_examples.html)

In [0]:
SET enable_result_cache_for_session TO OFF;

EXPLAIN
SELECT
  *
FROM sales
JOIN listing USING (listid)
LIMIT 10

In [0]:
SELECT
  *
FROM sales, listing 
WHERE sales.listid = listing.listid 
LIMIT 10

## DISTKEY - Speed Up 

DS_DIST_NONE (Distribution: None) which means that no redistribution is required. This is because corresponding slices are co-located on the compute nodes because they had the same DISTKEY. The cost ahs also decreased from 124168336397.54 to 18127.08.

In [0]:
CREATE TABLE sales_distkey
DISTKEY (listid)
AS
SELECT * FROM sales;

CREATE TABLE listing_distkey
DISTKEY (listid)
AS
SELECT * FROM listing;

In [0]:
SET enable_result_cache_for_session TO OFF;

EXPLAIN
SELECT
  *
FROM sales_distkey
JOIN listing_distkey USING (listid)
LIMIT 10

## Dimension Table 

DS_DIST_ALL_NONE (Distribution: None so no broadcast) which means that no redistribution was required because the inner join table used the default distribution of DISTSTYLE ALL. The entire table is located on every node. Notice that the query costs are very low because no information needed to be transferred between nodes.

In [0]:
EXPLAIN
SELECT
  *
FROM listing_distkey
JOIN dwdate USING (dateid)
LIMIT 10

## Data Distribution Check

In [0]:
SELECT * FROM svv_table_info
WHERE "table" in ('dwdate', 'sales', 'listing', 'sales_distkey', 'listing_distkey')

## Sort Key 

In the lab 3, there is a big_sale and sorted_big_salde tables with 353 millions rows.

In [0]:
SELECT *
FROM orders
ORDER BY o_orderdate DESC
LIMIT 20

In [0]:
-- explain
SELECT SUM(o_totalprice) AS revenue 
FROM orders 
WHERE o_orderdate BETWEEN '1998-08-02'::TIMESTAMP AND '1998-08-02'::TIMESTAMP;

In [0]:
CREATE TABLE sorted_orders
SORTKEY(o_orderdate)
AS 
SELECT * FROM orders;

In [0]:
-- explain
SELECT SUM(o_totalprice) AS revenue 
FROM sorted_orders 
WHERE o_orderdate BETWEEN '1998-08-02'::TIMESTAMP AND '1998-08-02'::TIMESTAMP;